In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import pathlib
import pysam
import pyfaidx
import warnings
import re
import argparse
import sys
import subprocess
from tqdm import tqdm 

tqdm.pandas()
warnings.filterwarnings("ignore")

main_outptudir = "/workdir/outdir/pipeline_output"

analysis_output_version = "20250624"

name = "Vi"

path_to_main_output = os.path.join(main_outptudir, "data_analysis", name, analysis_output_version)


rundf = pd.read_excel("../panel_design/All_panel_designs_20250601.xlsx", sheet_name="Runs_tracking")
rundf.columns = ["Run", "SampleID", "UMI", "Panel_version", "Name", "amplicon_name", "serial"]

# umi_rundf = rundf[(rundf["UMI"] == "Yes") & (rundf["Name"] == name) ]
umi_rundf = rundf[(rundf["Name"] == name) ]

collect_all_sample_sheets = [item for item in pathlib.Path("../experiments/").glob("SampleSheet_batch_*.csv")]

samplesheet = pd.DataFrame()
for file in collect_all_sample_sheets:
    df = pd.read_csv(file, sep=",")
    df["batch"] = file.name.replace(".csv", "")
    samplesheet = pd.concat([samplesheet, df], ignore_index=True)

samplesheet.columns = ["filename", "FASTQ1", "FASTQ2", "batch"]
# samplesheet["Run"] = samplesheet["FASTQ1"].apply(lambda x: x.split("/")[5] if x.split("/")[5] != "2025" else x.split("/")[6])
samplesheet["Run"] = samplesheet["FASTQ1"].apply(lambda x: [item for item in x.split("/") if item.startswith("R")][0] )
samplesheet["SampleID"] = samplesheet["filename"].apply(lambda x: x.split("_")[0].split("-")[1])
print(f"All available runs: {", ".join(sorted(samplesheet['Run'].unique()))}")

##### R7489
# selected_run = "R7489"
# UMI = False
# umt_distance = 0
# serial = False

##### R7512
# selected_run = "R7512"
# UMI = False
# umt_distance = 0
# serial = True

##### R7519
selected_run = "R7519"
UMI = False
umt_distance = 0
serial = True


umi_rundf = umi_rundf[umi_rundf["Run"] == selected_run]
samplesheet = samplesheet[samplesheet["Run"] == selected_run]

path_to_02_output = os.path.join(path_to_main_output, "02_output", selected_run)
os.system(f"mkdir -p {path_to_02_output}")

All available runs: R7288, R7297, R7312, R7331, R7332, R7347, R7353, R7373, R7374, R7381, R7393, R7400, R7434, R7477, R7483, R7489, R7512, R7519


0

In [2]:
umi_rundf

,Run,SampleID,UMI,Panel_version,Name,amplicon_name,serial
808,R7519,TML1S1,No,Lung_panel version 1.1,Vi,L1,S1
809,R7519,TML1S2,No,Lung_panel version 1.1,Vi,L1,S2
810,R7519,TML1S3,No,Lung_panel version 1.1,Vi,L1,S3
811,R7519,TML1S4,No,Lung_panel version 1.1,Vi,L1,S4
812,R7519,TML1S5,No,Lung_panel version 1.1,Vi,L1,S5
813,R7519,TML1S6,No,Lung_panel version 1.1,Vi,L1,S6
814,R7519,TML1S7,No,Lung_panel version 1.1,Vi,L1,S7
815,R7519,TML2S1,No,Lung_panel version 1.1,Vi,L2,S1
816,R7519,TML2S2,No,Lung_panel version 1.1,Vi,L2,S2
817,R7519,TML2S3,No,Lung_panel version 1.1,Vi,L2,S3


In [3]:
samplesheet

,filename,FASTQ1,FASTQ2,batch,Run,SampleID
233,10-TML2S3_S7522-S7722,/mnt/FASTQ/2025_2/R7519/zatm/10-TML2S3_S7522-S...,/mnt/FASTQ/2025_2/R7519/zatm/10-TML2S3_S7522-S...,SampleSheet_batch_20250629,R7519,TML2S3
234,11-TML2S4_S7523-S7723,/mnt/FASTQ/2025_2/R7519/zatm/11-TML2S4_S7523-S...,/mnt/FASTQ/2025_2/R7519/zatm/11-TML2S4_S7523-S...,SampleSheet_batch_20250629,R7519,TML2S4
235,12-TML2S5_S7524-S7724,/mnt/FASTQ/2025_2/R7519/zatm/12-TML2S5_S7524-S...,/mnt/FASTQ/2025_2/R7519/zatm/12-TML2S5_S7524-S...,SampleSheet_batch_20250629,R7519,TML2S5
236,13-TML2S6_S7525-S7725,/mnt/FASTQ/2025_2/R7519/zatm/13-TML2S6_S7525-S...,/mnt/FASTQ/2025_2/R7519/zatm/13-TML2S6_S7525-S...,SampleSheet_batch_20250629,R7519,TML2S6
237,15-TML3S1_S7527-S7727,/mnt/FASTQ/2025_2/R7519/zatm/15-TML3S1_S7527-S...,/mnt/FASTQ/2025_2/R7519/zatm/15-TML3S1_S7527-S...,SampleSheet_batch_20250629,R7519,TML3S1
238,16-TML3S2_S7528-S7728,/mnt/FASTQ/2025_2/R7519/zatm/16-TML3S2_S7528-S...,/mnt/FASTQ/2025_2/R7519/zatm/16-TML3S2_S7528-S...,SampleSheet_batch_20250629,R7519,TML3S2
239,17-TML3S3_S7529-S7729,/mnt/FASTQ/2025_2/R7519/zatm/17-TML3S3_S7529-S...,/mnt/FASTQ/2025_2/R7519/zatm/17-TML3S3_S7529-S...,SampleSheet_batch_20250629,R7519,TML3S3
240,18-TML3S4_S7530-S7730,/mnt/FASTQ/2025_2/R7519/zatm/18-TML3S4_S7530-S...,/mnt/FASTQ/2025_2/R7519/zatm/18-TML3S4_S7530-S...,SampleSheet_batch_20250629,R7519,TML3S4
241,19-TML3S5_S7531-S7731,/mnt/FASTQ/2025_2/R7519/zatm/19-TML3S5_S7531-S...,/mnt/FASTQ/2025_2/R7519/zatm/19-TML3S5_S7531-S...,SampleSheet_batch_20250629,R7519,TML3S5
242,1-TML1S1_S7513-S7713,/mnt/FASTQ/2025_2/R7519/zatm/1-TML1S1_S7513-S7...,/mnt/FASTQ/2025_2/R7519/zatm/1-TML1S1_S7513-S7...,SampleSheet_batch_20250629,R7519,TML1S1


In [ ]:
rerun = True
if (os.path.isfile(os.path.join(path_to_02_output, f"UMI_runs_tracking_{selected_run}.xlsx")) == False) | (rerun  == True):
    umi_rundf["uniqueID"] = umi_rundf[["SampleID", "Run"]].apply(lambda x: f"{x[0]}_{x[1]}", axis=1)
    samplesheet["uniqueID"] = samplesheet[["SampleID", "Run"]].apply(lambda x: f"{x[0]}_{x[1]}", axis=1)

    final_umi_rundf = umi_rundf.merge(samplesheet.drop(["SampleID", "Run"], axis = 1), left_on = "uniqueID", right_on = "uniqueID")
    final_umi_rundf["UMI_cov_file"] = final_umi_rundf[["batch", "filename"]].apply(lambda x: os.path.join(
        main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_COV", f"{x[1]}.connor_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov"
    ), axis = 1)
    final_umi_rundf["nonUMI_cov_file"] = final_umi_rundf[["batch", "filename"]].apply(lambda x: os.path.join(
        main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_COV", f"{x[1]}.CutAdapt_R1_bismark_bt2_pe.bedGraph.gz.bismark.zero.cov"
    ), axis = 1)

    final_umi_rundf["check_nonUMI"] = final_umi_rundf["nonUMI_cov_file"].apply(lambda x: os.path.exists(x))

    final_umi_rundf["unmapped_bam"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_ALIGNMENT_UNMAPPED_BAM", f"{x[1]}_R1.UMIprocessed_bismark_bt2_pe.bam"), axis = 1
    )
    if UMI:
        final_umi_rundf["check_UMI"] = final_umi_rundf["UMI_cov_file"].apply(lambda x: os.path.exists(x))
    
        final_umi_rundf["connor_annotated"] = final_umi_rundf[["batch", "filename"]].apply(
            lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "CONNOR_UMI_OUTPUT", f"{x[1]}.connor.fully_annotated.bam"), axis = 1
        )
        final_umi_rundf["connor_fastq"] = final_umi_rundf[["batch", "filename"]].apply(
            lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "CONNOR_UMI_OUTPUT", f"{x[1]}.connor_R1.fastq.gz"), axis = 1
        )
        final_umi_rundf["umi_bam"] = final_umi_rundf[["batch", "filename"]].apply(
            lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_ALIGNMENT", f"{x[1]}.connor_R1_bismark_bt2_pe.sorted.bam"), axis = 1
        )
        available_columns = ["unmapped_bam", "connor_annotated", "connor_fastq", "umi_bam", "non_umi_bam" , "UMI_cov_file"]
    
    if UMI == False:
        available_columns = ["unmapped_bam", "non_umi_bam"]
    final_umi_rundf["non_umi_bam"] = final_umi_rundf[["batch", "filename"]].apply(
        lambda x: os.path.join(main_outptudir, x[0], f"UMT_DISTANCE_{umt_distance}", "BISMARK_ALIGNMENT", f"{x[1]}.CutAdapt_R1_bismark_bt2_pe.sorted.bam"), axis = 1
    )

    tqdm.pandas()
    for col in available_columns:
        print(f"working on column: {col}")
        def _get_line_count(x):
            check_file = os.path.exists(x)
            x_raw = x.replace(".sorted.bam", ".bam")
            if check_file == False:
                if os.path.exists(x_raw) == True: 
                    # print(f"sorted bam file is not available, but raw bam file is available, sorting and indexing it now...")
                    os.system(f"samtools sort -@ 15 {x_raw} -o {x}")
                    os.system(f"samtools index {x}")
                else:
                    # rint(f"File {x} does not exist, skipping line count for this file.")
                    return 0
            if ".bam" in x:
                result = subprocess.run(f"samtools view {x} -c", shell=True, capture_output=True, text=True)
            elif ".fastq.gz" in x:
                result = subprocess.run(f"zcat {x} | wc -l", shell=True, capture_output=True, text=True)
            else:
                result = subprocess.run(f"cat {x} | wc -l", shell=True, capture_output=True, text=True)
            return int(result.stdout.strip())
            
        final_umi_rundf[f"count_{col}"] = final_umi_rundf[col].progress_apply(_get_line_count)    
    # small size fastq
    final_umi_rundf["small_FASTQ_size"] = final_umi_rundf["FASTQ1"].apply(lambda x: "smallsize" in x)
    final_umi_rundf.groupby("small_FASTQ_size")["SampleID"].count()
    final_umi_rundf.to_excel(os.path.join(path_to_02_output, f"UMI_runs_tracking_{selected_run}.xlsx"), index=False)
else:
    print("reading in saved processed UMI runs tracking file...")
    final_umi_rundf = pd.read_excel(os.path.join(path_to_02_output, f"UMI_runs_tracking_{selected_run}.xlsx"))

working on column: unmapped_bam


100%|██████████| 23/23 [00:00<00:00, 56746.47it/s]


working on column: non_umi_bam


  0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
final_umi_rundf

# helper functions

In [ ]:
# panel_name = "Lung_panel version 1.1"
# cpgdf = pd.DataFrame()
# panel_name = panel_name.replace(" ", "_")
# paneldf = pd.read_excel("../panel_design/All_panel_designs_20250601.xlsx", sheet_name=panel_name)
# paneldf[f"{genome_version}_org"] = paneldf[genome_version].values
# paneldf["lengthF"] = paneldf["Forward Primer Seq"].apply(lambda x: len(x))
# paneldf["lengthR"] = paneldf["Reverse Primer Seq"].apply(lambda x: len(x))

def _remove_primer_coords(x, lenF, lenR):
    chrom = x.split(":")[0]
    start = int(x.split(":")[1].split("-")[0])
    end = int(x.split(":")[1].split("-")[1])
    start = start + lenF - 1
    end = end - lenR + 1
    return f"{chrom}:{start}-{end}"

def get_refseq(path_to_all_fa, chrom, start, end):
        refseq = pyfaidx.Fasta(os.path.join(path_to_all_fa, "{}.fa".format(chrom)))
        return(str.upper(refseq.get_seq(name = "{}".format(chrom), start = start, end = end).seq))

path_to_all_fa = "/home/hieunguyen/resources/hg19"
genome_version = "hg19"

##### get list of all real cpg for this panel
all_cpgdf = dict()
all_cpgdf_with_primer = dict()
all_paneldf = dict()
for panel_name in final_umi_rundf.Panel_version.unique():
    
    panel_name = panel_name.replace(" ", "_")
    paneldf = pd.read_excel("../panel_design/All_panel_designs_20250601.xlsx", sheet_name=panel_name)
    paneldf[f"{genome_version}_org"] = paneldf[genome_version].values
    paneldf["lengthF"] = paneldf["Forward Primer Seq"].apply(lambda x: len(x))
    paneldf["lengthR"] = paneldf["Reverse Primer Seq"].apply(lambda x: len(x))
    paneldf[genome_version] = paneldf[[genome_version, "lengthF", "lengthR"]].apply(lambda x:_remove_primer_coords(x[0], x[1], x[2]), axis = 1)

    def generate_cpgdf(region_col, paneldf):
        outputdf = pd.DataFrame()
        print(f"working on panel: {panel_name}, with {len(paneldf[region_col].unique())} regions")
        for region in paneldf[region_col].unique():
            region_name = paneldf[paneldf[region_col] == region]["Amplicon"].values[0]
            chrom = region.split(":")[0].replace("Chr", "chr").replace("chr", "")
            start = int(region.split(":")[1].split("-")[0])
            end = int(region.split(":")[1].split("-")[1])
            refseq = pyfaidx.Fasta(os.path.join(path_to_all_fa, "chr{}.fa".format(chrom)))
            refseq_at_region = str.upper(refseq.get_seq(name = "chr{}".format(chrom), start = start, end = end).seq)

            all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_region)]
            cpg_coords = [f"chr{chrom}:{item + start}-{item + start + 1}" for item in all_cpg_in_cluster]

            tmp_outputdf = pd.DataFrame(data = cpg_coords, columns = ["CpG"])
            tmp_outputdf["region"] = region
            tmp_outputdf["region_name"] = region_name
            outputdf = pd.concat([outputdf, tmp_outputdf], axis = 0)
        print(f"outputdf shape after adding region {region}: {outputdf.shape}")
        return outputdf
    
    cpgdf = generate_cpgdf(genome_version, paneldf)
    cpgdf_with_primer = generate_cpgdf(f"{genome_version}_org", paneldf)
    
    cpgdf = cpgdf[["region", "CpG", "region_name"]]
    all_cpgdf[panel_name] = cpgdf.copy()

    cpgdf_with_primer = cpgdf_with_primer[["region", "CpG", "region_name"]]
    all_cpgdf_with_primer[panel_name] = cpgdf_with_primer.copy()

    all_paneldf[panel_name] = paneldf.copy()

# for i in all_cpgdf.keys():
#     tmpdf1 = all_cpgdf[i].copy()
#     tmpdf2 = all_cpgdf_with_primer[i].copy()
#     assert tmpdf1.shape[0] == tmpdf2.shape[0], f"Error: {i} cpgdf and cpgdf_with_primer have different number of rows"


In [ ]:

def generate_cov(input_cov_file, panel_name, sampleid, outputdir):
    os.system(f"mkdir -p {outputdir}")
    if os.path.exists(input_cov_file) == False:
        print(f"Input coverage file {input_cov_file} does not exist, skipping...")
        covdf = pd.DataFrame(columns=["chrom", "start", "end", "meth_density", "countC", "countT", "seq", "strand", "CpG", "check_context", "amplicon"])
        status = "File not available"
    else:
        print(f"Processing {input_cov_file} for panel {panel_name}...")
        # get list of CpG which are not in primer regions
        cpgdf = all_cpgdf[panel_name]
        covdf = pd.read_csv(input_cov_file, header = None, sep = "\t")
        covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
        if covdf.shape[0] != 0:
            covdf = covdf[covdf["chrom"].isin(["chrX", "chrY", "chrMT", "chrM"]) == False]
            covdf = covdf[covdf["chrom"].str.contains("chrUn") == False]
            covdf = covdf[covdf["chrom"].str.contains("_") == False]
            covdf["seq"] = covdf[["chrom", "start"]].progress_apply(lambda x: get_refseq(path_to_all_fa= path_to_all_fa, 
                                                                    chrom = x[0], start = x[1], end = x[1] + 1), axis = 1)
            covdf["strand"] = covdf["seq"].apply(lambda x: "+" if x != "CG" else "-")
            covdf_raw = covdf.copy()
            covdf["start"] = covdf[["seq", "start"]].apply(lambda x: x[1] + 1 if x[0] != "CG" else x[1], axis = 1)

            covdf["chrom"] = covdf["chrom"].apply(lambda x: str(x))
            covdf["CpG"] = covdf[["chrom", "start"]].apply(lambda x: f"{str(x[0])}:{x[1]}-{x[1] + 1}", axis = 1)
            covdf["check_context"] = covdf["CpG"].apply(lambda x: "CpG_context" if x in cpgdf["CpG"].values else "False")
            covdf["amplicon"] = covdf["CpG"].apply(lambda x: cpgdf[cpgdf["CpG"] == x]["region_name"].values[0] if x in cpgdf["CpG"].values else "NA")
            status = "Processed"
        else:
            print(f"Coverage file {input_cov_file} is empty, skipping...")
            covdf = pd.DataFrame(columns=["chrom", "start", "end", "meth_density", "countC", "countT", "seq", "strand", "CpG", "check_context", "amplicon"])
            status = "Empty file"
        covdf.to_excel(os.path.join(outputdir, f"{sampleid}.xlsx"))
    return status, os.path.join(outputdir, f"{sampleid}.xlsx")

# mode = "UMI"  # or "ignore_UMI" 
for mode in ["UMI", "ignore_UMI"]:
    all_status = []
    all_finished_path = []
    for j in range(final_umi_rundf.shape[0]):
        panel_name = final_umi_rundf.Panel_version.values[j].replace(" ", "_")
        run = final_umi_rundf.Run.values[j]
        sampleid = final_umi_rundf.SampleID.values[j]
        if mode == "UMI":
            input_cov_file = final_umi_rundf.UMI_cov_file.values[j]
        elif mode == "ignore_UMI":
            input_cov_file = final_umi_rundf.nonUMI_cov_file.values[j]

        outputdir = os.path.join(path_to_02_output, "cov", run, panel_name, mode)
        status, output_cov_path = generate_cov(input_cov_file=input_cov_file, panel_name=panel_name, sampleid=sampleid, outputdir=outputdir)
        all_status.append(status)
        all_finished_path.append(output_cov_path)
    final_umi_rundf[f"{mode}_status"] = all_status
    final_umi_rundf[f"{mode}_processed_cov"] = all_finished_path

In [ ]:
# run = "R7288"
# mode = "UMI"  # or "ignore_UMI"

if serial:
    for run in final_umi_rundf.Run.unique():
    # for run in ["R7400"]:
        if UMI == True:
            input_modes = ["UMI", "ignore_UMI"]
        else:
            input_modes = ["ignore_UMI"]
        for mode in input_modes:
            if mode == "UMI":
                plot_metadata = final_umi_rundf[(final_umi_rundf["Run"] == run) & 
                                                (final_umi_rundf["UMI_status"] == "Processed")].sort_values(by = "SampleID")
                selected_cov = "UMI_processed_cov"
            else:
                plot_metadata = final_umi_rundf[(final_umi_rundf["Run"] == run) & 
                                                (final_umi_rundf["ignore_UMI_status"] == "Processed")].sort_values(by = "SampleID")
                selected_cov = "ignore_UMI_processed_cov"

            run_panels = plot_metadata.Panel_version.unique()

            # check how many panels are available for this run
            print(f"Run {run} has {len(plot_metadata.Panel_version.unique())} panel(s), {', '.join(run_panels)}")

            for panel_name in run_panels:
                panel_name = panel_name.replace(" ", "_")
                cpgdf = all_cpgdf[panel_name].copy()
                paneldf = all_paneldf[panel_name].copy()

                for amplicon in plot_metadata[plot_metadata["amplicon_name"].isna() == False]["amplicon_name"].unique():
                    tmp_metadata = plot_metadata[plot_metadata["amplicon_name"] == amplicon].sort_values(by = "serial")
                    if "," in amplicon:
                        covdf = pd.DataFrame(data = cpgdf[cpgdf["region_name"].isin(amplicon.replace(" ", "").split(","))]["CpG"].values, columns = ["CpG"])
                    else:
                        covdf = pd.DataFrame(data = cpgdf[cpgdf["region_name"] == amplicon]["CpG"].values, columns = ["CpG"])
                        
                    os.system(f"mkdir -p {os.path.join(path_to_02_output, 
                                                    "S1_to_S5_in_amplicons", 
                                                    run,
                                                    panel_name, 
                                                    mode)}")
                    if tmp_metadata.shape[0] > 1:
                        if "," in amplicon:
                            amplicon = amplicon.replace(",", "-").replace(" ", "")
                        for i in range(tmp_metadata.shape[0]):
                            p = tmp_metadata.iloc[i][selected_cov]
                            sample  = tmp_metadata.iloc[i]["serial"]
                            tmp_covdf = pd.read_excel(p)[["CpG", "meth_density"]]
                            tmp_covdf.columns = ["CpG", f"{amplicon}_{sample}"]
                            covdf = covdf.merge(tmp_covdf, right_on = "CpG", left_on = "CpG")
                        if covdf.shape[0] > 1:
                            plt.figure(figsize=(12, 6))
                            covdf.set_index("CpG").T.plot.line()
                            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
                            plt.xticks(rotation=90)
                            plt.tight_layout()
                            plt.title(f"Methylation Density in {amplicon} Amplicon, {mode} \n Samples: {', '.join(tmp_metadata['serial'].unique())}")
                            plt.savefig(os.path.join(path_to_02_output, 
                                                    "S1_to_S5_in_amplicons", 
                                                    run,
                                                    panel_name, 
                                                    mode,
                                                    f"{amplicon}_methylation_density.png"),
                                                    dpi=300, 
                                                    bbox_inches='tight')

                            ######
                            # corr_plotdf = covdf.set_index("CpG").mean(axis = 0).reset_index()
                            # corr_plotdf.columns = ["CpG", "mean_meth_density"]
                            # corr_plotdf["true_meth"] = [0, 5, 10, 50, 100]

                            # plt.figure(figsize=(6, 6))
                            # sns.scatterplot(data = corr_plotdf, x = "true_meth", y = "mean_meth_density")
                            # corr = corr_plotdf[["true_meth", "mean_meth_density"]].corr().iloc[0,1]
                            # plt.text(60, 10, f"Pearson r = {corr:.2f}", fontsize=14)
                            # plt.plot([0, 100], [0, 100], 'r--')
                            # plt.xlabel("Expected methylation S1-S5 (%)")
                            # plt.ylabel("Observed mean methylation density")
                            # plt.title("Correlation between true and observed methylation (%)")
                            # plt.tight_layout()
                            # plt.savefig(os.path.join(path_to_02_output, 
                            #                         "S1_to_S5_in_amplicons", 
                            #                         run,
                            #                         panel_name, 
                            #                         mode,
                            #                         f"{amplicon}_correlation.png"), 
                            #                         dpi=300, 
                            #                         bbox_inches='tight')
                            # plt.show()
